In [1]:
import numpy as np
import requests
import gzip
import os
import hashlib
import numpy as np

def fetch(url):
    fp = os.path.join("/tmp", hashlib.md5(url.encode("utf-8")).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            dat = f.read()
    else:
        with open(fp, "wb") as f:
            dat = requests.get(url).content
            f.write(dat)
    return np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()

def fetch_mnist():
    X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28*28))
    Y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
    X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28*28))
    Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]
    return X_train, Y_train, X_test, Y_test

In [2]:
X_train, Y_train, X_test, Y_test = fetch_mnist()

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange


In [5]:
class ModifiedLeNet5(nn.Module):
    def __init__(self, input_shape=(1, 32, 32), classes=10):
        super(ModifiedLeNet5, self).__init__()

        # Layer 1
        self.convolution_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu_1 = nn.ReLU()
        self.batchnorm_1 = nn.BatchNorm2d(32)
        
        # Layer 2
        self.convolution_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=0, bias=False)
        
        # Layer 3
        self.batchnorm_2 = nn.BatchNorm2d(32)
        self.relu_2 = nn.ReLU()
        self.max_pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout_1 = nn.Dropout(0.25)
        
        # Layer 4
        self.convolution_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        
        # Layer 5
        self.convolution_4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0, bias=False)
        self.batchnorm_3 = nn.BatchNorm2d(64)
        
        # Layer 6
        self.relu_3 = nn.ReLU()
        self.max_pool_2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout_2 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        
        # Layer 7
        self.fully_connected_1 = nn.Linear(64*6*6, 256, bias=False)
        self.batchnorm_4 = nn.BatchNorm1d(256)
        
        # Layer 8
        self.relu_4 = nn.ReLU()
        
        # Layer 9
        self.fully_connected_2 = nn.Linear(256, 128, bias=False)
        self.batchnorm_5 = nn.BatchNorm1d(128)
        
        # Layer 10
        self.relu_5 = nn.ReLU()
        
        # Layer 11
        self.fully_connected_3 = nn.Linear(128, 84, bias=False)
        self.batchnorm_6 = nn.BatchNorm1d(84)
        
        # Layer 12
        self.relu_6 = nn.ReLU()
        self.dropout_3 = nn.Dropout(0.25)
        
        # Output Layer
        self.output = nn.Linear(84, classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.convolution_1(x)
        x = self.relu_1(x)
        x = self.batchnorm_1(x)

        x = self.convolution_2(x)

        x = self.batchnorm_2(x)
        x = self.relu_2(x)
        x = self.max_pool_1(x)
        x = self.dropout_1(x)

        x = self.convolution_3(x)

        x = self.convolution_4(x)
        x = self.batchnorm_3(x)

        x = self.relu_3(x)
        x = self.max_pool_2(x)
        x = self.dropout_2(x)
        x = self.flatten(x)

        x = self.fully_connected_1(x)
        x = self.batchnorm_4(x)
        x = self.relu_4(x)

        x = self.fully_connected_2(x)
        x = self.batchnorm_5(x)
        x = self.relu_5(x)

        x = self.fully_connected_3(x)
        x = self.batchnorm_6(x)
        x = self.relu_6(x)
        x = self.dropout_3(x)

        x = self.output(x)
        x = self.softmax(x)

        return x

# Instantiate the model
input_shape = (1, 32, 32)
num_classes = 10
model = ModifiedLeNet5(input_shape=input_shape, classes=num_classes)

In [ ]:
opt = torch.optim.Adam(model.parameters())